In [2]:
import pandas as pd
#import seaborn as sns
import numpy as np
#import matplotlib.pyplot as plt
import glob
from netCDF4 import Dataset
import json
import xarray as xr
#%matplotlib inline

In [3]:
save_dir = '/home/hanna/images_simple_statsmodels/'

In [4]:
coord_dir = '/home/hanna/Desktop/examples_master_thesis/'

In [5]:
coords_fil = glob.glob(coord_dir + '*.nc')[0]

In [6]:
coords_fil

'/home/hanna/Desktop/examples_master_thesis/W_XX-EUMETSAT-Darmstadt,SING+LEV+SAT,MSG4+CLM_C_EUMG_20190612113000.nc'

In [7]:
lat_lon = Dataset(coords_fil)

In [8]:
lat_array = lat_lon.variables['lat']
lon_array = lat_lon.variables['lon']

In [9]:
nc_files = glob.glob(save_dir+'*cell*.json')

In [10]:
nc_files

['/home/hanna/images_simple_statsmodels/changes_lat_lon_cell-area.json']

In [11]:
def add(a, b):
    return np.abs(a) + np.abs(b)

def subtract(a, b):
    return np.abs(a) - np.abs(b)

In [12]:
def read_dlon_dlat(save_dir):
    nc_files = glob.glob(save_dir+'*cell*.json')
    print(nc_files)
    
    with open(nc_files[-1]) as f:
        d =  json.load(f)
        
    d_phi      = d['dphi'] 
    d_theta    = d['dtheta'] 
    cell_areas = d['cell_area']
    lat_array  = d['lat'] 
    lon_array  = d['lon']
    #save_dict_to_json(d, "changes_lat_lon_cell-area.json", save_dir = save_dir)
    return d_phi, d_theta, cell_areas, lat_array, lon_array

In [13]:
d_phi, d_theta, cell_areas, lat_array, lon_array = read_dlon_dlat(save_dir)

['/home/hanna/images_simple_statsmodels/changes_lat_lon_cell-area.json']


In [14]:
def area_grid_cell(c_lat, d_lat, d_lon):
        """
        c_lat : float
            Centre point longitude, latitude in degrees

        d_lat, d_lon : float
            delta lat lon in degrees

        Returns : area in km^2

        cdo : If the grid cell area have to be computed it is scaled with the earth radius to square meters.
        """
        R = 6371000  # in M
        # area er egentlig R**2
        area = R*(np.sin((c_lat + d_lat)*np.pi/180) - np.sin((c_lat - d_lat)*np.pi/180) )*(2*d_lon*np.pi/180) # R**2
        return np.abs(area)


In [15]:
lats = np.arange(30, 56, 0.25)  # numpy split these.
lons = np.arange(-15, 30, 0.25) # numpy split these 

c_lat = np.array(lat_array)
c_lon = np.array(lon_array) 
d_theta = np.array(d_theta) 
d_phi = np.array(d_phi)

write_fil = True
era_step = 0.25

lats = [52.5]
lons = [27.5]

data = {}
counter = 0
for lat in lats:
    data[str(lat)] = {}
    counter+=1
    for lon in lons:
        data[str(lat)][str(lon)] = {}
        era_AREA = area_grid_cell(lat, era_step/2, era_step/2)
        print("lat = {}, lon = {}".format(lat, lon))
        era_up    = lat + era_step/2
        era_down  = lat - era_step/2
        era_left  = lon - era_step/2
        era_right = lon + era_step/2

        # to ensure correct sign
        cmk_left  = c_lon - np.abs(d_phi)   #- era_right
        cmk_right = c_lon + np.abs(d_phi)   #- era_left

        # TODO : Sjekk d theta og lignende.
        cmk_up    = c_lat + np.abs(d_theta) #- era_down
        cmk_down  = c_lat - np.abs(d_theta) #- era_up

        data[str(lat)][str(lon)]['centre'] = {}
        data[str(lat)][str(lon)]['down'] = {}
        data[str(lat)][str(lon)]['up'] = {}
        data[str(lat)][str(lon)]['right'] = {}
        data[str(lat)][str(lon)]['left'] = {}
        data[str(lat)][str(lon)]['corner'] = {}

        idx_left_boundary  = np.intersect1d(np.argwhere(cmk_right > era_left),  np.argwhere(cmk_left < era_left) )
        idx_right_boundary = np.intersect1d(np.argwhere(cmk_right > era_right), np.argwhere(cmk_left < era_right) )
        idx_up_boundary    = np.intersect1d(np.argwhere(cmk_up    > era_up),    np.argwhere(cmk_down < era_up) )
        idx_down_boundary  = np.intersect1d(np.argwhere(cmk_up    > era_down),  np.argwhere(cmk_down < era_down) )

        idx_lower_right_corner = np.intersect1d(idx_down_boundary, idx_right_boundary)
        idx_lower_left_corner  = np.intersect1d(idx_down_boundary, idx_left_boundary)
        idx_upper_left_corner  = np.intersect1d(idx_up_boundary,   idx_left_boundary)
        idx_upper_right_corner = np.intersect1d(idx_up_boundary,   idx_right_boundary)

        corner_idx = np.concatenate([idx_lower_right_corner, idx_lower_left_corner,
                                     idx_upper_left_corner, idx_upper_right_corner]) 
        
        data[str(lat)][str(lon)]['corner']['index'] = corner_idx.reshape(-1).tolist()

        # TODO add new centre corners. And calculate  
        llc_dlat = subtract(cmk_up[idx_lower_left_corner], era_down)/2
        llc_dlon = subtract(era_left, cmk_right[idx_lower_left_corner])/2
        llc_lat  = era_down + llc_dlat
        llc_area = area_grid_cell(llc_lat, llc_dlat, llc_dlon)

        lrc_dlat = subtract(cmk_up[idx_lower_right_corner], era_down)/2
        lrc_dlon = subtract(cmk_left[idx_lower_right_corner], era_right)/2
        lrc_lat  = era_down + lrc_dlat
        lrc_area = area_grid_cell(lrc_lat, lrc_dlat, lrc_dlon)

        ulc_dlat = subtract(era_up, cmk_down[idx_upper_left_corner])/2
        ulc_dlon = subtract(era_left, cmk_right[idx_upper_left_corner])/2
        ulc_lat  = era_up - ulc_dlat
        
        ulc_area = area_grid_cell(ulc_lat, ulc_dlat, ulc_dlon)

        urc_dlat = subtract(era_up, cmk_down[idx_upper_right_corner])/2
        urc_dlon = subtract(cmk_left[idx_upper_right_corner], era_right)/2
        urc_lat  = era_up - urc_dlat
        
        urc_area = area_grid_cell(urc_lat, urc_dlat, urc_dlon)

        corner_areas = np.array([lrc_area, llc_area, ulc_area, urc_area])
        data[str(lat)][str(lon)]['corner']['area'] = np.concatenate(corner_areas).reshape(-1).tolist()

        # Removes Corners
        # TODO test if this removes indecies. 
        idx_down_boundary = np.array(idx_down_boundary)[np.isin(idx_down_boundary, corner_idx, invert = True)]
        idx_up_boundary = np.array(idx_up_boundary)[np.isin(idx_up_boundary, corner_idx, invert = True)]
        idx_left_boundary = np.array(idx_left_boundary)[np.isin(idx_left_boundary, corner_idx, invert = True)]
        idx_right_boundary = np.array(idx_right_boundary)[np.isin(idx_right_boundary, corner_idx, invert = True)]    
        #idx_down_boundary = np.array(idx_down_boundary)[np.isin(idx_down_boundary, corner_idx invert = True)]
                       
        # subsection left boundary OLD
        low_bound = np.argwhere(cmk_down[idx_left_boundary] < era_up  )
        up_bound  = np.argwhere(cmk_up[idx_left_boundary]   > era_down )
        sub_section_left = np.intersect1d(low_bound, up_bound)
        idx_l = np.array(idx_left_boundary)[np.array(sub_section_left)]
        idx_left = idx_l[np.isin(idx_l, corner_idx, invert = True)].tolist()
        data[str(lat)][str(lon)]['left']['index'] = idx_left

        # subsection right boundary
        low_bound = np.argwhere( cmk_down[idx_right_boundary] < era_up )
        up_bound  = np.argwhere( cmk_up[idx_right_boundary] > era_down)
        sub_section_right = np.intersect1d(low_bound, up_bound)

        idx_r = np.array(idx_right_boundary)[np.array(sub_section_right)]
        idx_right = idx_r[np.isin(idx_r, corner_idx, invert = True)].tolist()
        data[str(lat)][str(lon)]['right']['index'] = idx_right

        # Subsection Down Boundary
        one = np.argwhere(cmk_left[idx_down_boundary]  > era_left)
        two = np.argwhere(cmk_right[idx_down_boundary] < era_right)
        sub_section_down = np.intersect1d(one, two)
        idx_d = np.array(idx_down_boundary)[np.array(sub_section_down)]
        idx_down = idx_d[np.isin(idx_d, corner_idx, invert = True)].tolist()
        data[str(lat)][str(lon)]['down']['index'] =  idx_down
        #sub_section_down.reshape(-1).tolist()

        # subsection up boundary
        one = np.argwhere( cmk_left[idx_up_boundary]  > era_left)
        two = np.argwhere( cmk_right[idx_up_boundary] < era_right)
        sub_section_up = np.intersect1d(one, two)
        print(sub_section_up)
        idx_u = np.array(idx_up_boundary)[np.array(sub_section_up)]
        idx_up = idx_u[np.isin(idx_u, corner_idx, invert = True)].tolist()
        data[str(lat)][str(lon)]['up']['index'] = idx_up

        # test that these are empty
        t1 = np.intersect1d(idx_d, idx_u)
        t2 = np.intersect1d(idx_r, idx_l)
        # assert len(t1) == len(t2) == 0, "intercept up down  {}, intersect left right {}.".format(t1, t2)

        if len(t1) != 0:
            print("\n Problem up, down lat {} lon {}. \n".format(lat, lon))


        if len(t2) != 0:
            print("\n Problem left, right lat {} lon {}. \n".format(lat, lon))
        # Calculate Boundaries

        # AREA left boundary
        dlon_lf = subtract(cmk_right[idx_left], era_left)/2 
        dlat_lf = d_theta[idx_left]
        lat_lf  = c_lat[idx_left]
        left_areas = area_grid_cell(lat_lf, dlat_lf, np.abs(dlon_lf))
        data[str(lat)][str(lon)]['left']['area'] = left_areas.reshape(-1).tolist()

        # AREA right boundary -- her er problemet !!!!!!!!
        dlon_rb = subtract(era_right, cmk_left[idx_right])/2
        dlat_rb = d_theta[idx_right]
        lat_rb  = c_lat[idx_right]
        right_area = area_grid_cell(lat_rb, dlat_rb, np.abs(dlon_rb)) 
        data[str(lat)][str(lon)]['right']['area'] = right_area.ravel().tolist()

        # AREA down boundary
        dlat_down = subtract(era_down, cmk_up[idx_down])/2
        lat_down = era_down + dlat_down
        dlon_down = d_phi[idx_down]
        down_area = area_grid_cell(lat_down, dlat_down, dlon_down)
        data[str(lat)][str(lon)]['down']['area'] = down_area.reshape(-1).tolist()
 
        # AREA up
        dlat_up = subtract(era_up, cmk_down[idx_up])/2
        lat_up = era_up - dlat_up
        #lon_up = c_lon[idx_up_boundary][sub_section_up]
        dlon_up = d_phi[idx_up]
        up_area = area_grid_cell(lat_up, dlat_up, np.abs(dlon_up))
        data[str(lat)][str(lon)]['up']['area'] = up_area.reshape(-1).tolist()
        # = np.sum(up_area)

        # Index centres. Can safly assume centre cells are correct.
        idx_centre_one = np.intersect1d(np.argwhere(cmk_left  > era_left),
                                        np.argwhere(cmk_right < era_right))


        idx_centre_two = np.intersect1d(np.argwhere(cmk_up   < era_up),
                                        np.argwhere(cmk_down >  era_down))

        idx_centre_cells = np.intersect1d( idx_centre_one, idx_centre_two )
        data[str(lat)][str(lon)]['centre']['index'] = idx_centre_cells.ravel().tolist()

        lat_centre_cells = c_lat[idx_centre_cells]
        dlat_centre      = d_theta[idx_centre_cells]
        dlon_centre      = d_phi[idx_centre_cells]
        
        centre_area      = area_grid_cell(lat_centre_cells, dlat_centre, dlon_centre)

        data[str(lat)][str(lon)]['centre']['area'] = centre_area.reshape(-1).tolist()

    #filnavn = 'ERA5_grid_resample_includes_changes_to_indexes_from_{}_{}.json'.format(float(lat)-5, lat)
    #if lat%5==0 and lat > 30:
    #    print("Saves file every lat {} lon {}.".format(lat, lon))
    filnavn = 'ERA5_grid_resample_test_right_boundary_{}_{}.json'.format(float(lat), lon)
    with open(save_dir + filnavn, 'w') as f:
        json.dump(data, f)
    #data = {}

#filnavn = 'ERA5_grid_resample_includes_lat{}_lon{}.json'.format(lat, lon)
#with open(save_dir + filnavn, 'w') as f:
#    json.dump(data[str(lat)][str(lon)], f)
#data = {}        


lat = 52.5, lon = 27.5


/home/hanna/anaconda3/envs/sciclouds/lib/python3.6/site-packages/ipykernel_launcher.py:44: RuntimeWarning: invalid value encountered in greater
/home/hanna/anaconda3/envs/sciclouds/lib/python3.6/site-packages/ipykernel_launcher.py:44: RuntimeWarning: invalid value encountered in less
/home/hanna/anaconda3/envs/sciclouds/lib/python3.6/site-packages/ipykernel_launcher.py:45: RuntimeWarning: invalid value encountered in greater
/home/hanna/anaconda3/envs/sciclouds/lib/python3.6/site-packages/ipykernel_launcher.py:45: RuntimeWarning: invalid value encountered in less
/home/hanna/anaconda3/envs/sciclouds/lib/python3.6/site-packages/ipykernel_launcher.py:46: RuntimeWarning: invalid value encountered in greater
/home/hanna/anaconda3/envs/sciclouds/lib/python3.6/site-packages/ipykernel_launcher.py:46: RuntimeWarning: invalid value encountered in less
/home/hanna/anaconda3/envs/sciclouds/lib/python3.6/site-packages/ipykernel_launcher.py:47: RuntimeWarning: invalid value encountered in greater
/

In [16]:
print(data)

{'52.5': {'27.5': {'centre': {'index': [12604649, 12604650, 12604651, 12608360, 12608361, 12608362, 12608363, 12612072, 12612073, 12612074, 12615783, 12615784, 12615785], 'area': [4.188149258453053, 4.193815026918698, 4.199217043244294, 4.196536272247417, 4.202271147366084, 4.207874328008221, 4.21341034037004, 4.21076716926, 4.216435184082651, 4.222108472073302, 4.219367232976641, 4.224972081200086, 4.230778341051892]}, 'down': {'index': [12600938, 12600939, 12600940], 'area': [0.7629532365066963, 1.07453645580619, 1.3876369724046786]}, 'up': {'index': [], 'area': []}, 'right': {'index': [12604652, 12608364, 12612075], 'area': [3.8065038338790944, 0.5221556303025091, 1.4387007683424107]}, 'left': {'index': [12604648, 12608359, 12612071, 12615782], 'area': [1.5082267203817468, 0.5676240854604291, 3.8359329170775758, 2.9115155070030343]}, 'corner': {'index': [12600941, 12600937, 12615786], 'area': [1.1650089377177455, 0.26617004725968824, 2.2060012633788837]}}}}


In [17]:
filnavn = 'ERA5_grid_resample_includes_lat{}_lon{}.json'.format(lat, lon)
with open(save_dir + filnavn, 'w') as f:
    json.dump(data, f)
#data = {}

In [18]:
data

{'52.5': {'27.5': {'centre': {'index': [12604649,
     12604650,
     12604651,
     12608360,
     12608361,
     12608362,
     12608363,
     12612072,
     12612073,
     12612074,
     12615783,
     12615784,
     12615785],
    'area': [4.188149258453053,
     4.193815026918698,
     4.199217043244294,
     4.196536272247417,
     4.202271147366084,
     4.207874328008221,
     4.21341034037004,
     4.21076716926,
     4.216435184082651,
     4.222108472073302,
     4.219367232976641,
     4.224972081200086,
     4.230778341051892]},
   'down': {'index': [12600938, 12600939, 12600940],
    'area': [0.7629532365066963, 1.07453645580619, 1.3876369724046786]},
   'up': {'index': [], 'area': []},
   'right': {'index': [12604652, 12608364, 12612075],
    'area': [3.8065038338790944, 0.5221556303025091, 1.4387007683424107]},
   'left': {'index': [12604648, 12608359, 12612071, 12615782],
    'area': [1.5082267203817468,
     0.5676240854604291,
     3.8359329170775758,
     2.91151550

In [19]:
data[str(lat)][str(lon)]

{'centre': {'index': [12604649,
   12604650,
   12604651,
   12608360,
   12608361,
   12608362,
   12608363,
   12612072,
   12612073,
   12612074,
   12615783,
   12615784,
   12615785],
  'area': [4.188149258453053,
   4.193815026918698,
   4.199217043244294,
   4.196536272247417,
   4.202271147366084,
   4.207874328008221,
   4.21341034037004,
   4.21076716926,
   4.216435184082651,
   4.222108472073302,
   4.219367232976641,
   4.224972081200086,
   4.230778341051892]},
 'down': {'index': [12600938, 12600939, 12600940],
  'area': [0.7629532365066963, 1.07453645580619, 1.3876369724046786]},
 'up': {'index': [], 'area': []},
 'right': {'index': [12604652, 12608364, 12612075],
  'area': [3.8065038338790944, 0.5221556303025091, 1.4387007683424107]},
 'left': {'index': [12604648, 12608359, 12612071, 12615782],
  'area': [1.5082267203817468,
   0.5676240854604291,
   3.8359329170775758,
   2.9115155070030343]},
 'corner': {'index': [12600941, 12600937, 12615786],
  'area': [1.1650089377

In [21]:
#import simplejson as json

In [22]:
def save(data, filnavn):
    #filnavn = 'ERA5_grid_resample_counter{}_lat{}_lon{}.json'.format(counter, lat, lon)
    with open(save_dir + filnavn, 'w') as f:
        json.dump(data, f)
#data = {}

In [23]:
for key, item in data.items():
    for key2, item2 in item.items():
        #save(item2, "a.json")
        for key3, item3 in item2.items():
            #save(item3, "b.json")
            for key4, item4 in item3.items():
                #save(item4, "c.json")
                print(item4)

[12604649, 12604650, 12604651, 12608360, 12608361, 12608362, 12608363, 12612072, 12612073, 12612074, 12615783, 12615784, 12615785]
[4.188149258453053, 4.193815026918698, 4.199217043244294, 4.196536272247417, 4.202271147366084, 4.207874328008221, 4.21341034037004, 4.21076716926, 4.216435184082651, 4.222108472073302, 4.219367232976641, 4.224972081200086, 4.230778341051892]
[12600938, 12600939, 12600940]
[0.7629532365066963, 1.07453645580619, 1.3876369724046786]
[]
[]
[12604652, 12608364, 12612075]
[3.8065038338790944, 0.5221556303025091, 1.4387007683424107]
[12604648, 12608359, 12612071, 12615782]
[1.5082267203817468, 0.5676240854604291, 3.8359329170775758, 2.9115155070030343]
[12600941, 12600937, 12615786]
[1.1650089377177455, 0.26617004725968824, 2.2060012633788837]


In [24]:
type(item)

dict

In [ ]:
import json

In [ ]:
filnavn = 'ERA5_grid_resample_counter{}_lat{}_lon{}.json'.format(counter, lat, lon)
with open(save_dir + filnavn, 'w') as f:
    json.dump(item, f)

In [ ]:
glob.glob(save_dir + '*json')

In [ ]:
counter